In [1]:
import ete3
import os
import re
import multiprocessing
import pickle as pkl
import linecache
import pandas as pd
import numpy as np
import random
import plotly
import plotly.plotly as ptl
from plotly import graph_objs as go
import pyparsing as pp

plotly_accession = open('/Users/thiberio/plotly_accession').read().split()
ptl.sign_in(plotly_accession[0], plotly_accession[1])

os.chdir('/work/Alphas_and_Cyanos')

class cd:
    """
    Context manager for changing the current working directory
    """
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [2]:
newick = open('/work/Alphas_and_Cyanos/ranger_input_trees-no_long_branches/002440.tree').read()

In [124]:
#https://github.com/bxlab/bx-python/blob/master/lib/bx/phylo/newick.py
#https://stackoverflow.com/questions/51373300/how-to-convert-newick-tree-format-to-a-tree-like-hierarchical-object
ident = pp.Word(pp.hexnums)
value = pp.pyparsing_common.real

label = pp.Word(pp.hexnums)
value = pp.pyparsing_common.real
real  = pp.Combine(pp.Word( "+-" + pp.nums, pp.nums ) +
                  pp.Optional( "." + pp.Optional( pp.Word( pp.nums ) ) ) +
                  pp.Optional( pp.CaselessLiteral( "E" ) + pp.Word( "+-" + pp.nums, pp.nums ) ) )

lpar  = pp.Suppress( "(" )
rpar  = pp.Suppress( ")" )
colon = pp.Suppress( ":" )
coma  = pp.Suppress( "," )

taxa   = pp.Group(label + pp.Optional(colon + real) + pp.Optional(coma))
parser = pp.OneOrMore(pp.nestedExpr(content=pp.delimitedList(taxa)))

tests = """
    ((A:0.1,B:0.2)5(C:0.3,D:0.4)98)F:0.9
"""
parsed_results = parser.parseString(tests)
import pprint
pprint.pprint(parsed_results.asList(), width=20)

[[[['A', '0.1'],
   ['B', '0.2']],
  ['5'],
  [['C', '0.3'],
   ['D', '0.4']],
  ['98']]]


In [241]:
#https://github.com/bxlab/bx-python/blob/master/lib/bx/phylo/newick.py
#https://stackoverflow.com/questions/51373300/how-to-convert-newick-tree-format-to-a-tree-like-hierarchical-object
label = pp.Word(pp.alphas+'_')
value = pp.pyparsing_common.real
real  = pp.Combine(pp.Word( "+-" + pp.nums, pp.nums ) +
                  pp.Optional( "." + pp.Optional( pp.Word( pp.nums ) ) ) +
                  pp.Optional( pp.CaselessLiteral( "E" ) + pp.Word( "+-" + pp.nums, pp.nums ) ) )

lpar  = pp.Suppress( "(" )
rpar  = pp.Suppress( ")" )
colon = pp.Suppress( ":" )
coma  = pp.Suppress( "," )

taxa    = pp.Group(label + colon + real)
support = pp.Group(pp.Optional(real + colon) + real + pp.Optional(coma))

parser = pp.OneOrMore(pp.nestedExpr(content=pp.delimitedList(taxa)) | support)

tests = "(A:0.1,B:0.2)5,(C:0.3,D:0.4)98)F:0.9"
parsed_results = parser.parseString(newick)
pprint.pprint(parsed_results.asList(), width=20)

ParseException: Expected {nested () expression | Group:({[{Combine:({W:(+-01...,0123...) [{"." [W:(0123...)]}] [{'E' W:(+-01...,0123...)}]}) Suppress:(":")}] Combine:({W:(+-01...,0123...) [{"." [W:(0123...)]}] [{'E' W:(+-01...,0123...)}]}) [Suppress:(",")]})} (at char 1), (line:1, col:2)

In [247]:
len(pp.nestedExpr().parseString(newick).asList()[0])

5

In [243]:
pprint.pprint(pp.nestedExpr().parseString(newick).asList(), width=20)

[['GCA000010525_BAF86964:0.33056416210000000300,',
  ['GCA000022085_ACL58092:0.30563727239999999297,GCA000017645_ABS66721:0.46352064560000000304'],
  '99:0.07552300000000000679,',
  [[['GCA000632475_AIB16204:0.27924899749999998466,',
     [['GCA000092285_ADG12505:0.29220938099999999027,GCA002005445_AQS85952:0.45235732350000001922'],
      '90:0.04998099999999999765,',
      ['GCA000264455_AFK56038:0.00997550699999999978,GCA001601315_KYO52468:0.00603114109999999988'],
      '100:0.43698300000000001031'],
     '71:0.03959399999999999725'],
    '68:0.04137799999999999812,',
    [['GCA900177295_SMF82384:0.00000266990000000000,GCA900188055_SNS27395:0.00000266990000000000'],
     '100:0.32553199999999998804,',
     [[[[['GCA000013085_ABC24271:0.39760138360000002455,GCA000143145_ADJ22320:0.59637580619999996934'],
         '21:0.04007200000000000345,',
         ['GCA000019845_ACB94614:0.49489940119999997759,',
          [[[['GCA000018545_ACP23037:0.45499483759999997945,GCA002043005_AQZ52212:0.

In [149]:
taxa = pp.Forward()
taxa   << pp.Group(pp.Optional(pp.OneOrMore(lpar)) + pp.Optional(coma) + label + colon + real + (coma|lpar))
taxa.parseString(tests)

([(['A', '0.1'], {})], {})

In [189]:
def Syntax():
    label = pp.Word(pp.hexnums)
    value = pp.pyparsing_common.real
    real  = pp.Combine(pp.Word( "+-" + pp.nums, pp.nums ) +
                      pp.Optional( "." + pp.Optional( pp.Word( pp.nums ) ) ) +
                      pp.Optional( pp.CaselessLiteral( "E" ) + pp.Word( "+-" + pp.nums, pp.nums ) ) )

    lpar  = pp.Suppress( "(" )
    rpar  = pp.Suppress( ")" )
    colon = pp.Suppress( ":" )
    coma  = pp.Suppress( "," )

    
    expr = pp.Forward()
    node = pp.Optional(pp.OneOrMore(lpar) | coma) + \
           pp.Group(label + colon + real) + \
           pp.Optional(rpar | coma) + \
           pp.Optional(real)
    expr <<= node + pp.ZeroOrMore(node)
    return expr

if __name__ == "__main__":
    expr = Syntax()
    def test(s):
        results = expr.parseString(s)
        print (s,'->', results)

    test( "((A:0.1,B:0.2)5(C:0.3,D:0.4)98)F:0.9" )

((A:0.1,B:0.2)5(C:0.3,D:0.4)98)F:0.9 -> [['A', '0.1'], ['B', '0.2'], '5', ['C', '0.3'], ['D', '0.4'], '98']


<h3>Functions!</h3>

In [ ]:
def match_rooting(ref_tree, tree2):
    tree_to_root = tree2.copy()
    for node in sorted( ref_tree.children, key=len ):
        if node.is_leaf():
            leaf = tree_to_root.get_leaves_by_name(node.name)[0]
            tree_to_root.set_outgroup(leaf)
            break
        else:
            is_it_monophyletic, clade_type, fucking_up = tree_to_root.check_monophyly(node.get_leaf_names(), 'name', unrooted=False)
            if is_it_monophyletic:
                equivalent = tree_to_root.get_common_ancestor(node.get_leaf_names())
                tree_to_root.set_outgroup(equivalent)
            else:
                tree_to_root.set_outgroup(fucking_up.pop())
                equivalent = tree_to_root.get_common_ancestor(node.get_leaf_names())
                tree_to_root.set_outgroup(equivalent)
            break

    return tree_to_root

In [ ]:
def rename_branches(reconciliation_file, tree):
    branches         = re.findall('^(m\d+) = LCA\[(\S+), (\S+)\]:', reconciliation_file, re.M)
    duplicated_names = {}
    for name, leaf1, leaf2 in branches:
        node = tree.get_common_ancestor(leaf1, leaf2)
        if node.name:
            duplicated_names[name] = node.name
            continue
        node.name = name
    return tree, duplicated_names

In [ ]:
def parse_aggregated(folder, threshold=0.9, leaves_allowed=False):
    if not os.path.isdir(folder) or not os.path.isfile(
            '/work/Alphas_and_Cyanos/aggregated/mad_roots-stricter_branch_lengths/%s' % folder):
        return {folder:None}

    aggregated = open(
        '/work/Alphas_and_Cyanos/aggregated/mad_roots-stricter_branch_lengths/%s' % folder).read()
    with cd(folder):
        gene_tree     = {'named':ete3.Tree(linecache.getline('%s-MAD.ranger_out1' %folder, 8), format=1)}

    gene_tree['support']        = root_like(gene_tree['named'], ete3.Tree('/work/Alphas_and_Cyanos/ranger_input_trees-no_long_branches/%s.tree' %folder))
    gene_tree, duplicated_names = rename_branches(aggregated, gene_tree['support'])

    ufboot_distribution = [node.support for node in gene_tree.traverse() if not node.is_leaf()]
    if np.percentile(ufboot_distribution, 25) < 80:
        return {folder:[[], gene_tree]}

    num_replicates = float(re.match('Processed (\d+) files', aggregated).group(1))

    if not leaves_allowed:
        transfers = re.findall('^(m\d+) = .*, Transfers = [^0]\d+?\], \[Most Frequent mapping --> (n\d+), (\d+) times\], \[Most Frequent recipient --> (n\d+), (\d+) times\].', aggregated, re.M)
    else:
        transfers = re.findall('^(m\d+) = .*, Transfers = [^0]\d+?\], \[Most Frequent mapping --> (\S+), (\d+) times\], \[Most Frequent recipient --> (\S+), (\d+) times\].',   aggregated, re.M)

    supported_transfers = []
    for donor_map, donor, ranger_confidence_donor, recipient, ranger_confidence_recipient in transfers:
        if int(ranger_confidence_donor) < threshold*num_replicates or int(ranger_confidence_recipient) < threshold*num_replicates:
            continue
        supported_transfers.append((donor_map, donor, recipient))

    selected_transfers = []
    for donor_map_name, donor_name, recipient_name in supported_transfers:
        if donor_map_name in duplicated_names:
            donor_map = gene_tree.search_nodes(name=duplicated_names[donor_map_name])[0]
        else:
            donor_map = gene_tree.search_nodes(name=donor_map_name)[0]
        if donor_map.support < 95:
            continue

        recipient_map_search = re.search('^({children[0]}|{children[1]}).*Most Frequent mapping --> {recipient}'.format(
            recipient=recipient_name,
            children=[child.name for child in donor_map.children]),
            aggregated, re.M)
        if recipient_map_search:
            recipient_map_name = recipient_map_search.group(1)
            if not all([donor_name, recipient_name, donor_map_name, recipient_map_name]):
                continue
            selected_transfers.append({'donor':donor_name, 'recipient':recipient_name, 'donor_map':donor_map_name, 'recipient_map':recipient_map_name})

    return {folder:[selected_transfers, gene_tree]}

In [ ]:
with cd('reconciliations/mad_roots-stricter_branch_lengths'):
    pool    = multiprocessing.Pool(processes=15)
    results = pool.map(parse_aggregated, os.listdir('.')[:10])
    pool.close()
    pool.join()

    transfers = {}
    for filtered in results:
        if  list(filtered.values()) != [[]] and list(filtered.values())[0][0] != []:
            transfers.update(filtered)

out = open('aggregated/mad_transfers-test.pkl', 'wb')
pkl.dump(transfers, out)
out.close()

out = open('aggregated/maxtic.constrains-test', 'w')
for group, (transfer_data, gene_tree) in transfers.items():
    for transfer in transfer_data:
        out.write('%s\t%s\n' % (transfer['donor'], transfer['recipient']))
out.close()

In [ ]:
filtered

In [ ]:
list(filtered.values())[0][0]